In [ ]:
%pylab inline

In [ ]:
import sacc
import lmfit
import emcee

In [ ]:
%cd ../../

In [ ]:
from lss_like import LSSLikelihood

In [ ]:
from lss_theory import LSSTheory

In [ ]:
p = lmfit.Parameters()
p.add_many(('omega_c', 0.3, True, 1e-3,0.7), ('omega_b', 0.05,True,1e-4,0.1), ('h0', 0.8,True,0.1,2), ('sigma_8', 0.8, False),('n_s',0.97,False),
           ('omega_k',0,False),('omega_nu',0,False),('w0',-1.0,False),
          ('wa',0.0,False),('des_gals_z_b_1',0.05,False),('des_gals_b_b_1',1.,True,0.1,10.),('des_gals_z_b_2',0.5,False),
           ('des_gals_b_b_2',1.,True,0.1,10.),('des_gals_z_b_3',2.,False),('des_gals_b_b_3',1.,True,0.1,10))

In [ ]:
ll=LSSLikelihood('sim_sample/sims/sim_mean.sacc')

In [ ]:
theory = LSSTheory('sim_sample/sims/sim_mean.sacc')

In [ ]:
#This is particular to LMFit. The parameters cannot be vectors
#I had to create an intermediate step to fiddle with the bias
#This is kind of slow because it has to open the data file 3 times we should optimize this
#Also ccl is utterly slow...
def loglike(params,data_file):
    ll=LSSLikelihood(data_file)
    s=sacc.SACC.loadFromHDF(data_file)
    dic_par = params.valuesdict()
    for tr in s.tracers :
            if tr.type == 'point' :
                dic_par[tr.exp_sample+'_z_b']=[]
                dic_par[tr.exp_sample+'_b_b']=[]
                for pkey in dic_par.keys():
                    if tr.exp_sample+'_z_b_' in pkey:
                        dic_par[tr.exp_sample+'_z_b'].append(dic_par[pkey])
                    if tr.exp_sample+'_b_b_' in pkey:
                        dic_par[tr.exp_sample+'_b_b'].append(dic_par[pkey])
    theory = LSSTheory(data_file)
    return ll(theory.get_prediction(dic_par))

In [ ]:
loglike(p,'sim_sample/sims/sim_mean.sacc')

In [ ]:
mini = lmfit.Minimizer(loglike,p,fcn_args=[theory,ll])
res = mini.emcee(burn=10, steps=100, thin=10, params=p)